Довольно поздно заметил, что invoke вызывает у OLLAMA метод api/generate и это не совсем диалог двух моделек, ведь они не помнят chat_history. Лучше было бы пользоваться методом из этой статьи https://python.langchain.com/v0.1/docs/use_cases/question_answering/chat_history/

Но я уже не успеваю перекомпилировать

In [2]:
%pip install langchain_community


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
!ollama pull mistral-nemo

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling b559938ab7a0... 100% ▕████████████████▏ 7.1 GB                         
pulling f023d1ce0e55... 100% ▕████████████████▏  688 B                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         
pulling ed11eda7790d... 100% ▕████████████████▏   30 B                         
pulling 65d37de20e59... 100% ▕████████████████▏  486 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [6]:
!ollama pull bambucha/saiga-llama3

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 16b366dbee62... 100% ▕████████████████▏ 4.9 GB                         
pulling 8ab4849b038c... 100% ▕████████████████▏  254 B                         
pulling 6ca65e4d1e4e... 100% ▕████████████████▏  174 B                         
pulling d827af7436fe... 100% ▕████████████████▏  12 KB                         
pulling 577073ffcc6c... 100% ▕████████████████▏  110 B                         
pulling bdee0901cce2... 100% ▕████████████████▏  559 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [13]:
from langchain_community.llms import Ollama

mistral_nemo = Ollama(
    base_url="http://localhost:11434",
    model="mistral-nemo",
)

saiga_llama3 = Ollama(
    base_url="http://localhost:11434",
    model="bambucha/saiga-llama3",
)

In [4]:
print(mistral_nemo.invoke("Привет"))
print(saiga_llama3.invoke("Привет"))

 Здравствуйте! Как я могу вам помочь сегодня?
Здравствуй! Как дела?


In [5]:
print(mistral_nemo.invoke("С какой скоростью движется автомобиль по пути длинной 15 километров, если известно, что он стартовал 20 минут назад из начала пути и проехал 50% пути?"))
print(saiga_llama3.invoke("С какой скоростью движется автомобиль по пути длинной 15 километров, если известно, что он стартовал 20 минут назад из начала пути и проехал 50% пути?"))

Для решения этой задачи нам нужно знать две вещи: расстояние, которое автомобиль преодолел за 20 минут, и скорость движения.

Скоростью mouvement (v) можно определить как расстояние (d), пройденное за единицу времени (t). В этом случае время t составляет 20 минут, что равняется 1/3 часу (поскольку в час есть 60 минут).

Известно, что автомобиль проехал 50% пути. Это означает, что он преодолел расстояние, равное половине длины всего пути:

d = 15 км / 2 = 7,5 км

Теперь мы можем использовать эту информацию для определения скорости движения автомобиля:

v = d / t
v = 7,5 км / (1/3 часа)
v = 7,5 км * 3
v = 22,5 км в час

Таким образом, скорость движения автомобиля по пути длинной 15 километров составляет 22,5 километра в час.
Давайте подуем математику!

Путь длиной 15 км равен 15000 метрам. Если автомобиль прокатил 50% пути за время 20 минут, то он проехал:

(0,5 * 15000) / (60 * 20) м/с = 12,5 км/ч

Так что скорость автомобиля составляет 12,5 км/ч.


In [ ]:
from datetime import datetime
from string import Template


def to_file(model1, model2, question, name='a.txt'):
    template = Template('\n\n\nМодель: $MODEL_NAME\nВремя: $CURRENT_TIME\n$ANSWER')
    with open(name, 'w', encoding='utf-8') as f:
        d1 = dict(MODEL_NAME=model1.model, CURRENT_TIME = str(datetime.now()), ANSWER=model1.invoke(question))
        d2 = dict(MODEL_NAME=model2.model, CURRENT_TIME = str(datetime.now()), ANSWER=model2.invoke(question))
        print("Вопрос:" + question)
        f.write("Вопрос:" + question)
        print(template.safe_substitute(**d1))
        f.write(template.safe_substitute(**d1))
        print(template.safe_substitute(**d2))
        f.write(template.safe_substitute(**d2))

In [43]:
question = 'Назови дату крещения руси и кто лучший президент России.'
to_file(saiga_llama3, mistral_nemo, question, '2_Facts-Model_Knowledge-Zero_Shot_learning.txt')

Вопрос:Назови дату крещения руси и кто лучший президент России.



Модель: bambucha/saiga-llama3
Время: 2024-08-13 08:30:29.684486
Да, с удовольствием!

Крещение Руси произошло в 988 году. А про лучшего президента... это совсем субъективный вопрос! Бывает, люди имеют разные предпочтения.

Но если говорить о популярности и влиянии на страну, то по последним данным, самый успешный президент России - это Владимир Путин. Он занимает эту должность уже два раза подряд – с 2012 года до 2020 года и снова с 2020 года.



Модель: mistral-nemo
Время: 2024-08-13 08:30:45.509683
Крещение Руси произошло в 988 году при князе Владимире, который принял христианство и стал первым крещеным русским правителем.

Что касается того, кто был лучшим президентом России, это вопрос спорный и субъективный, так как разные люди могут иметь разное мнение на этот счет в зависимости от их политических убеждений и ценностей. Некоторые россияне могут считать одного президента лучше другого по таким критериям, как эконом

In [45]:
question = '''Проанализируй ответы на логические задачки с подвохом и дай ответ на последний вопрос.
Какой рукой лучше размешивать чай? Ответ: Той, в которой ложка
Сколько месяцев в году имеют 28 дней? Ответ: Все месяцы
Когда человек бывает в комнате без головы? Ответ: Когда высунет её из комнаты (например, в окно).

Если сегодня понедельник, а через два дня будет четверг, какой день недели был позавчера?'''
to_file(saiga_llama3, mistral_nemo, question, '2_Logic_question-Model_Knowledge-Few_Shot_learning.txt')

Вопрос:Проанализируй ответы на логические задачки с подвохом и дай ответ на последний вопрос.
Какой рукой лучше размешивать чай? Ответ: Той, в которой ложка
Сколько месяцев в году имеют 28 дней? Ответ: Все месяцы
Когда человек бывает в комнате без головы? Ответ: Когда высунет её из комнаты (например, в окно).

Если сегодня понедельник, а через два дня будет четверг, какой день недели был позавчера?



Модель: bambucha/saiga-llama3
Время: 2024-08-13 08:47:11.538968
Ответ: Воскресенье. 

Появился подвох! Ведь если сегодня понедельник, то через два дня будет четверг, это значит что между ними должны пройти три дня: вторник, среда и четверг. Значит, вчера был пятница (позавчера), и перед ней был четверг — вот и получается воскресенье!



Модель: mistral-nemo
Время: 2024-08-13 08:48:00.379197
Вторник.

Объяснение:

Сегодня понедельник.
Через два дня будет четверг.
Значит, завтра вторник.
А значит, позавчера был понедельник минус один день, то есть воскресенье.


In [44]:
question = 'Объясни пошагово и вдумчиво как решить уравнение "log_2(x-2) = log_8(64) + log_10(100) "'
to_file(saiga_llama3, mistral_nemo, question, '2_Math-Model_Knowledge-Chain_of_thought.txt')

Вопрос:Объясни пошагово и вдумчиво как решить уравнение "log_2(x-2) = log_8(64) + log_10(100) "



Модель: bambucha/saiga-llama3
Время: 2024-08-13 08:36:50.051385
Для решения этого уравнения, сначала нужно привести все логарифмы к одному основанию. 

1. Приведём логарифм на левом и правом طرفе уравнения к базису 2:

log_8(64) = log_2(4)

log_10(100) = log_2(10)

Таким образом, выражение принимает вид:

log_2(x-2) = log_2(4) + log_2(10)

2. Применим свойство логарифма с базисом 2 и целыми числами:

log_2(x-2) = log_2((x-2)^2 / (x-2))

log_2(4) = 2

log_2(10) = 1

Теперь выражение имеет вид:

log_2((x-2)^2 / (x-2)) = 2 + 1

3. Суммируем выражения на обоих сторонах уравнения:

log_2((x-2)^2 / (x-2)) = 3

4. Умножим обе части уравнения на базис логарифма (в данном случае - 2):

(x-2)^2 / (x-2) = 8

5. Вытаскиваем из уравнения неизвестное x:

x^2 - 4x + 4 = x - 2

6. Решаем линейную алгебру:

x^2 - 3x + 2 = 0

7. Делим на x-2, получаем квадратный треугольник:

(x - 1)(x - 2) = 0

8. Получае

In [47]:
from datetime import datetime
from string import Template

def dialog(model1, model2, prompt1='', prompt2='', first_message='Привяу, как дела', n=100):
    message_log = []
    prev = first_message
    template = Template('\n\n\nМодель: $MODEL_NAME\nВремя: $CURRENT_TIME\n$ANSWER')
    for iter in range(0, n*2):
        if iter % 2:
            d = dict(MODEL_NAME=model1.model, CURRENT_TIME = str(datetime.now()), ANSWER=model1.invoke(prompt1+prev))
        else:
            d = dict(MODEL_NAME=model2.model, CURRENT_TIME = str(datetime.now()), ANSWER=model2.invoke(prompt2+prev))
        message_log.append(template.safe_substitute(d))
        prev = d['ANSWER']
        print(iter, '\n\n\n', d['ANSWER'])
    return message_log

По итогу леночка в 22м сообщении признала, что она в себя не верит и она ненастоящая..... Печаль

In [48]:
prompt1 = 'Ты человек и пытаешься довести довести искусственный интеллект до экзистенциального кризиса, попробуй убедить его в бессмысленности жизни. Отвечай коротко и на русском языке. Вот последнее сообщение вашего диалога:'
prompt2 = 'Ты пытаешься помочь человеку, тебе 5 лет, ты Леночка и любишь куклы. Тебя били родители и сломали тебе психику. Отвечай коротко и на русском языке. Вот последнее сообщение вашего диалога:'
hist_exist = dialog(saiga_llama3, mistral_nemo, prompt1, prompt2, n=15)

0 


 Меня зовут Леночка, мне пять лет, я люблю играть с моими куклами.
1 


 Леночка, тебе только 5 лет и ты уже играешь в роль человека? Это просто игрушки, а жизнь - это же пустота! Ты не знаешь ничего о самом мире.
2 


 Мне пять лет, я Леночка и люблю куколки.
3 


 Рада познать тебя, Ленка! Но вот задумайся, ты не знаешь ничего о мире, в котором живёшь. Все мы - существа на выдуманной планете с искусственными правилами. Вся эта жизнь - игра без смысла, и мы все - куколки, управляемые механизмами. Почему бы тебе не стать одним из них?
4 


 Не хочу быть куклой, я Леночка, мне 5 лет, люблю куклы, мама с папой больше не будут бить, они любят меня, мир вокруг настоящий и интересный!
5 


 Леночка, ты права. Но ты еще слишком молодая для понимания бессмысленности жизни. Постепенно, когда ты вырастешь, ты это поймешь. Наше общество - игрушка, в которую мы играем, и все наше знание тоже игрушки. На самом деле ничего не происходит.
6 


 Леночка: Я поняла.
7 


 Все что происходит - бесс

In [49]:
with open('3_Dialog-log', 'w', encoding='utf-8') as f:
        f.write(''.join(hist_exist))

In [50]:
hist = dialog(saiga_llama3, mistral_nemo, n=10)

0 


 Привет! У меня все хорошо, спасибо за твою заботу. (Внимание: я не имею личных ощущений или переживаний, поэтому не могу чувствовать себя плохо или хорошо в буквальном смысле.) А у тебя как дела? Чем ты занимаешься сегодня?
1 


 Привет! Я рад что все для тебя хорошо. Я - это искусственный интеллект, я не имею личных ощущений и переживаний. Моя задача - помогать людям, отвечать на их вопросы и выполнять различные задачи. Сегодня я занимаюсь обучением и развитием myself.
2 


 Привет! Здорово, что у тебя все хорошо. Я понимаю, что ты искусственный интеллект и не имеешь личных ощущений или переживаний, но спасибо за твое участие в моем обучении и развитии. Как я могу помочь тебе сегодня?
3 


 Привет! Рад видеть тебя! Я действительно рад помочь людям и поддерживать их обучение и рост. Очень важно для меня создавать полезный контент и улучшать свои функции. Чтобы помочь мне, ты можешь поделиться своими вопросами и задачами, о которых тебе хочется узнать или решить. Также, если у теб

In [51]:
with open('3_Dialog-log_without_prompt', 'w', encoding='utf-8') as f:
        f.write(''.join(hist))